In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Motivation/Problem-Statement" data-toc-modified-id="Motivation/Problem-Statement-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Motivation/Problem Statement</a></div><div class="lev2 toc-item"><a href="#Code-requirements" data-toc-modified-id="Code-requirements-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Code requirements</a></div><div class="lev2 toc-item"><a href="#Exposition-/-other-notebooks" data-toc-modified-id="Exposition-/-other-notebooks-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Exposition / other notebooks</a></div><div class="lev2 toc-item"><a href="#Boilerplate-code-for-representing-and-manipulating-probability-distributions" data-toc-modified-id="Boilerplate-code-for-representing-and-manipulating-probability-distributions-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Boilerplate code for representing and manipulating probability distributions</a></div><div class="lev1 toc-item"><a href="#Developing-a-model-of-a-BSC-with-input-signals-of-varying-length-and-a-'uniphone'-noise/channel-distribution" data-toc-modified-id="Developing-a-model-of-a-BSC-with-input-signals-of-varying-length-and-a-'uniphone'-noise/channel-distribution-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Developing a model of a BSC with input signals of varying length and a 'uniphone' noise/channel distribution</a></div><div class="lev2 toc-item"><a href="#Defining-p(X_0^f)-and-related-input-pmfs" data-toc-modified-id="Defining-p(X_0^f)-and-related-input-pmfs-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Defining $p(X_0^f)$ and related input pmfs</a></div><div class="lev3 toc-item"><a href="#Convenient-values/functions-defiend-in-terms-of-p(X_0^f)-(prefixes,-suffixes,-etc.)" data-toc-modified-id="Convenient-values/functions-defiend-in-terms-of-p(X_0^f)-(prefixes,-suffixes,-etc.)-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Convenient values/functions defiend in terms of $p(X_0^f)$ (prefixes, suffixes, etc.)</a></div><div class="lev2 toc-item"><a href="#Defining-p(Y_j|x_j)-and-related-channel-pmfs" data-toc-modified-id="Defining-p(Y_j|x_j)-and-related-channel-pmfs-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Defining $p(Y_j|x_j)$ and related channel pmfs</a></div><div class="lev2 toc-item"><a href="#Defining-hat{p}(\widehat{x}_0^j|x_0^i)" data-toc-modified-id="Defining-hat{p}(\widehat{x}_0^j|x_0^i)-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Defining $\hat{p}(\widehat{x}_0^j|x_0^i)$</a></div><div class="lev1 toc-item"><a href="#Generating-test-lexicons-and-noise-distributions" data-toc-modified-id="Generating-test-lexicons-and-noise-distributions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generating test lexicons and noise distributions</a></div><div class="lev2 toc-item"><a href="#Generating-p(X_0^f)" data-toc-modified-id="Generating-p(X_0^f)-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Generating $p(X_0^f)$</a></div><div class="lev2 toc-item"><a href="#Generating-p(Y_i|x_i)" data-toc-modified-id="Generating-p(Y_i|x_i)-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Generating $p(Y_i|x_i)$</a></div><div class="lev1 toc-item"><a href="#Generating-model-code" data-toc-modified-id="Generating-model-code-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Generating model code</a></div><div class="lev2 toc-item"><a href="#Input/source-model..." data-toc-modified-id="Input/source-model...-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Input/source model...</a></div><div class="lev2 toc-item"><a href="#Channel-model..." data-toc-modified-id="Channel-model...-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Channel model...</a></div><div class="lev2 toc-item"><a href="#Total-model..." data-toc-modified-id="Total-model...-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Total model...</a></div><div class="lev2 toc-item"><a href="#Testing-the-generated-code-on-the-generated-lexicon-and-noise-distributions" data-toc-modified-id="Testing-the-generated-code-on-the-generated-lexicon-and-noise-distributions-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Testing the generated code on the generated lexicon and noise distributions</a></div>

**Notebook author:** emeinhardt@ucsd.edu

# Motivation/Problem Statement

The goal of this collection of notebooks ('Model Notebook k', $k \in \{0, 1, 2, 3\}$) is to document/develop code for calculations related to an isolated word recognition task where:

 - A *lexicon* $\mathcal{L}$ is a set of **wordforms**  $w$, where a **wordform** is a finite sequence $s_0, s_1, s_2 ... s_f$ of segments, and $s_0^i$ denotes the prefix of some wordform $w \in \mathcal{L}$ such that $f = |w| - 1$ and such that $w$ begins with segments $s_0, s_1, ..., s_i$, where $0 \leq i \leq f$.
 - In a single episode of the task, a **speaker** samples a wordform $w = s_0^f$ from $\mathcal{L} \sim p(s_0^f)$.
 - The speaker incrementally produces their intended wordform (one segment at a time, for our purposes) and the **listener** incrementally perceives $\sigma_0^i \sim p(\sigma_0^i | s_0^i)$.
 - The listener considers what they have perceived so far $\sigma_0^i$ and reasons about what the most likely actual intended wordform of the speaker is $\sim p(\hat{s_0^f}|\sigma_0^i) \propto p(\sigma_0^i|s_0^f) p(s_0^f) = p(\sigma_0^i|s_0^i) p(s_0^i)$.


In particular, we want, for each possible intended wordform $s_0^f \in \mathcal{L}$, for each possible prefix $s_0^i$ of $s_0^f$, the listener's expected distribution over the speaker's intended wordform given the actual prefix produced so far $s_0^i$, where the expectation is taken with respect to possible perceived sequences $\sigma_0^i$: 

$$p(\hat{s_0^f}|s_0^i) = \sum_\limits{\sigma_0^i} p(s_0^f|\sigma_0^i)p(\sigma_0^i|s_0^i) \propto \sum_\limits{\sigma_0^i} p(\sigma_0^i|\hat{s_0^f})p(\hat{s_0^f})p(\sigma_0^i|s_0^i)$$

## Code requirements

The code for calculating 
$$p(\hat{s_0^f}|s_0^i) = \sum_\limits{\sigma_0^i} p(s_0^f|\sigma_0^i)p(\sigma_0^i|s_0^i) \propto \sum_\limits{\sigma_0^i} p(\sigma_0^i|\hat{s_0^f})p(\hat{s_0^f})p(\sigma_0^i|s_0^i)$$
needs to take only two basic inputs:

 - a prior distribution over the lexicon $p(s_0^f) = p(\hat{s_0^f})$.
 - an incrementally defined channel distribution $p(\sigma_0^i|s_0^i) = p(\sigma_0^i|\hat{s_0^i})$, defined (for now) in terms of a uniphone channel distribution $p(\sigma_i|s_i)$, where the distribution over what segment the listener perceives as the $i$th one depends (by assumption) *only* on the actual $i$th segment $s_i$ the speaker produced (if they've actually produced it at the time we're asking about).

from which it needs to generate all relevant code and representations of probability distributions or means of estimating them.

## Exposition / other notebooks

*Notebook 0* introduces the model being developed and implemented, the organization of other notebooks in the collection, and introduces the adapted form of Peter Norvig's code for conveniently representing and manipulating probability distributions that I make use of in other notebooks.

*This notebook*, *Notebook 1* introduces the final model implementation and the derivations underlying its implementation and does so using randomly generated but highly constrained binary lexicons with a simple noise model. It is intended to introduce the model implementation/derivation in a context where the behavior and purpose (and basic correctness) of code is discernable by going through the notebook and the generated example.

*Notebook 2* is more for testing purposes. It contains multiple implementations of the model. One is defined entirely in terms of the abstractions Peter Norvig provides -- abstractions that cannot scale but which were easy to use and whose correctness I have high confidence in. The second implementation is the one presented in notebooks 1 and 3. 

*Notebook 3* is a demonstration notebook where real data is loaded and largely the same queries as here calculated, plus some timing experiments.

## Boilerplate code for representing and manipulating probability distributions

In [2]:
import random

#from 
#    http://nbviewer.jupyter.org/url/norvig.com/ipython/Probability.ipynb
#    http://nbviewer.jupyter.org/url/norvig.com/ipython/ProbabilityParadox.ipynb
#with slight modification.

from fractions import Fraction

from collections import defaultdict, Counter

is_predicate = callable

def P(event, space): 
    """The probability of an event, given a sample space of equiprobable outcomes. 
    event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs."""
    if is_predicate(event):
        event = such_that(event, space)
    if isinstance(space, ProbDist):
        return sum(space[o] for o in space if o in event)
    else:
        return Fraction(len(event & space), len(space))
    
def such_that(predicate, space): 
    """The outcomes in the sample pace for which the predicate is true.
    If space is a set, return a subset {outcome,...};
    if space is a ProbDist, return a ProbDist {outcome: frequency,...};
    in both cases only with outcomes where predicate(element) is true."""
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o)})
    else:
        return {o for o in space if predicate(o)}

# class ProbDist(dict):
class ProbDist(Counter):
    "A Probability Distribution; an {outcome: probability} mapping where probabilities sum to 1."
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        total = sum(self.values())
        if isinstance(total, int): 
            total = Fraction(total, 1)
        for key in self: # Make probabilities sum to 1.
            self[key] = self[key] / total
            
    def __and__(self, predicate): # Call this method by writing `probdist & predicate`
        "A new ProbDist, restricted to the outcomes of this ProbDist for which the predicate is true."
        return ProbDist({e:self[e] for e in self if predicate(e)})
    
    def __repr__(self):
        s = ""
        for k in self:
            if isinstance(self[k], Fraction):
                s+="{0}: {2}/{3} = {1}\n".format(transcriptionReprHack(k), float(self[k]), self[k].numerator, self[k].denominator)
            else:
                s+="{0}: {1}\n".format(transcriptionReprHack(k), float(self[k]))
        return s

dottedStringToTuple = lambda ds: tuple(ds.split('.'))
tupleToDottedString = lambda t: '.'.join(t)

def transcriptionReprHack(k):
    if type(k) == type(tuple()):
        if all(map(lambda el: type(el) == type(''), k)):
            return tupleToDottedString(k)
    return k.__repr__()    

def Uniform(outcomes): return ProbDist({e: 1 for e in outcomes})

def joint(A, B):
    """The joint distribution of two independent probability distributions. 
    Result is all entries of the form {(a, b): P(a) * P(b)}"""
    return ProbDist({(a,b): A[a] * B[b]
                    for a in A
                    for b in B})

from itertools import product
from functools import reduce
import operator

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def union(iterable):
    return reduce(set.union, iterable)

def joint2(iter_of_dists):
    #ProbDist({(a,b): A[a] * B[b] for a in A for b in B})
    #ProbDist({ab: A[ab[0]] * B[ab[1]] for ab in product(A,B)})
    return ProbDist({each : prod(dist[each[i]] for i,dist in enumerate(iter_of_dists)) for each in list(product(*iter_of_dists))})

In [3]:
#bookkeeping - don't worry about this function...
def getBoundaryVal(dist, i):
    #p = {a:1/2, b:1/2} -> 2 items -> 2-1=1 boundaries where 
    #                   the boundary separating 'a' (item 0) from 'b' (item 1) is at 0+p(a)
    #q = {0:1/3, 1:1/3, 2:1/6, 3:1/6} -> 4 items -> 4-1=3 boundaries where 
    #                   the boundary separating 0 from 1   is at 0 + q(0) = 1/3,
    #                   the boundary separating 1 from 2   is at 0 + q(0) + q(1) = 2/3,
    #                   the boundary separating 2 from 3   is at 0 + q(0) + q(1) + q(2) = 5/6,
    #                ...the boundary separating i from i+1 is at \sum_{j=0}^{j=i} q(j)
    outcomes = list(dist.keys())
    if i >= len(outcomes) - 1:
        raise Exception("Boundary i = {0} out of bounds / does not exist for distribution {1} with {2} outcomes.".format(i, dist, len(outcomes)))
    if i == 0:
        return dist[outcomes[0]]
    return dist[outcomes[i]] + getBoundaryVal(dist, i-1)

#bookkeeping - don't worry about this function...
def getSampleOutcomeIndex(randReal, boundariesLeft, currIndex):
#     print("boundariesLeft: {0}".format(boundariesLeft))
#     print("currIndex: {0}".format(currIndex))
    if boundariesLeft == [] or randReal <= boundariesLeft[0]:
        return currIndex
    return getSampleOutcomeIndex(randReal, boundariesLeft[1:], currIndex + 1)


def sampleFrom(dist, num_samples = None):
    """
    Given a distribution (either an {outcome: probability} mapping where the 
    probabilities sum to 1 or an implicit definition of a distribution via a thunk), 
    this returns a single sample from the distribution, unless num_samples is specified, 
    in which case a generator with num_samples samples is returned.
    """
    if num_samples == None:
        if callable(dist):
            return dist()
        elif isinstance(dist, ProbDist):
            outcomes = list(dist.keys())
        #     print("outcomes: {0}".format(outcomes))

            boundaries = [getBoundaryVal(dist, i) for i in range(len(outcomes)-1)]
        #     print("boundaries: {0}".format(boundaries))

            randVal = random.random() #random real from unit interval
        #     print("randval: {0}".format(randVal))

            sampledOutcomeIndex = getSampleOutcomeIndex(randVal, boundaries, 0)
        #     print("sampledOutcomeIndex: {0}".format(sampledOutcomeIndex))
            if not (sampledOutcomeIndex >= 0 and sampledOutcomeIndex < len(outcomes)):
                print('sampledOutcomeIndex: {0}'.format(sampledOutcomeIndex))
                print('len(outcomes): {0}'.format(len(outcomes)))
                if len(outcomes) == 0:
                    print('len(outcomes) == 0! dist:')
                    print(type(dist))
                    print(dist)
            assert(sampledOutcomeIndex >= 0 and sampledOutcomeIndex < len(outcomes))

            sampledOutcome = outcomes[sampledOutcomeIndex]
        #     print("sampledOutcome: {0}".format(sampledOutcome))
            return sampledOutcome
    else:
        if callable(dist):
            return (dist() for each in range(num_samples))
        elif isinstance(dist, ProbDist):
            return (sampleFrom(dist, num_samples = None) for each in range(num_samples))

from collections import Counter

def frequencies(samples):
    return Counter(samples)

def makeSampler(dist):
    """
    Given a ProbDist, returns a thunk that when called, returns one sample from dist.
    """
    return lambda: sampleFrom(dist)

In [4]:
leftEdge = '⋊'
rightEdge = '⋉'
edgeSymbols = set([leftEdge, rightEdge])

# Developing a model of a BSC with input signals of varying length and a 'uniphone' noise/channel distribution

This is a 'word recognition model' with a small lexicon over a tiny segment inventory ($\{0,1\}$) and a simple channel distribution.

In more detail, by "BSC with sequential input" I really mean something easy to program and similar to my eventual application/goal:
 - The sender picks a sequence $X_0^f$ (e.g. 'cigarette') from a finite list of input sequences according to a commonly known distribution $p(X_0^f)$
     - here, the list of sequences has somewhere between 2-5 sequences
     - here, each sequence is 1-5 symbols long (not including word boundary symbols)
       - Note that for $0 \leq i \leq f$, $x_i$ is *never* a word boundary symbol, but $x_{-1}$ and $x_{f+1}$ are the left and right word edge symbols.
     - here, each symbol in a given sequence is chosen by sampling from a distribution $p(X_i)$ over a single symbol (not including word boundary symbols)
     - here, $p(X_0^f)$ is the uniform distribution for ease of implementation
 - Here and in the uniphone noise model for word recognition, the channel output for the $ith$ symbol in any input sequence depends only on the actual $ith$ input symbol:
     - $p(y_0^i|x_0^i) = \prod_\limits{j=0}^{j=i} p(y_j|x_j)$
     - Word boundary symbols are assumed to always be perfectly transmitted.
 - As the production of the sender/speaker's intended sequence/wordform $x_0^f$ unfolds, only some prefix of it $x_0^i$ has actually been produced so far.
 - Given that $x_0^i$ has been produced/sent, the receiver/listener perceives some possibly noise-corrupted sequence (e.g. 'shigarette') and uses their beliefs about what signals/wordforms are more or less likely to be produced and how channel noise works to *estimate* what the speaker's intended wordform $\hat{x_0^f}$ is (or any prefix thereof $\hat{x_0^j}$) - e.g. 'shigarette' isn't a word of English, but 'cigarette' is and [s] can relatively easily be misheard (or misproduced) as [ʃ], so the speaker's most likely intended wordform is 'cigarette'.

Below is the exact equation for the receiver/listener's incremental degree of belief that the sender/speaker's intended input sequence is $\hat{x}_0^f$ given that the speaker has actually produced $x_0^i$ of their actual intended sequence/wordform $x_0^f$:

$$p(\hat{x_0^f}|x_0^i) = \sum_\limits{y_0^i} p(\hat{x}_0^f|y_0^i)p(y_0^i|x_0^i) = \sum_\limits{y_0^i} \frac{p(y_0^i|\hat{x_0^f})p(\hat{x_0^f})}{p(y_0^i)}p(y_0^i|x_0^i) = \sum_\limits{y_0^i} \frac{p(y_0^i|\hat{x_0^i})p(\hat{x_0^f})}{p(y_0^i)}p(y_0^i|x_0^i) $$
where 
 - $p(\hat{X}_0^f) = p(X_0^f)$
 - $p(y_0^i|\hat{x}_0^i) = p(y_0^i|x_0^i) = \prod_\limits{j=0}^{j=i} p(y_j|x_j)$, and where $p(y_j|x_j)$ doesn't depend on $j$ / is independent of $j$ given $x_j$.
 - $p(y_0^i) = \sum_\limits{x_0^i} p(y_0^i|x_0^i)p(x_0^i)$

The MC estimator $\hat{p}(\hat{x}_0^f|x_0^i)$ is calcualted as follows:

$$\hat{p}(\hat{x_0^f}|x_0^i) = \frac{1}{n} \sum_\limits{n \text{ samples from } p(Y_0^i|x_0^i)}  p(\hat{x}_0^f|y_0^i) = \frac{1}{n} \sum_\limits{n \text{ samples from } p(Y_0^i|x_0^i)} \frac{p(y_0^i|\hat{x_0^f})p(\hat{x_0^f})}{p(y_0^i)} = \frac{1}{n} \sum_\limits{n \text{ samples from } p(Y_0^i|x_0^i)} \frac{p(y_0^i|\hat{x_0^i})p(\hat{x_0^f})}{p(y_0^i)}$$

In summary, the two key defining objects are $p(X_0^f)$ (defined via $p(X_i)$ and a procedure for constructing lexicons from $p(X_i)$) and $p(Y_i|X_i)$, and I'll be using them to calculate:
 - $p(y_0^i|x_0^i)$ and a function for sampling from $p(Y_0^i|x_0^i)$
 - $p(y_0^i)$
 - a Monte Carlo estimator $\hat{p}(\widehat{x}_0^j|x_0^i)$

## Defining $p(X_0^f)$ and related input pmfs

In [10]:
#defining p(X_0^f) as a dist

bits = set([0, 1])

useEdgeSymbols = True
leftEdge = '⋊'
rightEdge = '⋉'
edgeSymbols = set([leftEdge, rightEdge])

# p(X_i)
inputDist_i = Uniform(bits)

# function that samples from p(X_i)
channelInput_i = makeSampler(inputDist_i)


#define support of p(X_0^f), where 0 ≤ f ≤ 4...
def generateInputSequence(x_i_sampler):
    length = sampleFrom(Uniform(range(1,5)))
    return tuple(x_i_sampler() for each in range(length))

def padInputSequenceWithBoundaries(inputSeq):
  temp = list(inputSeq)
  temp = tuple([leftEdge] + temp + [rightEdge])
  return temp

def trimBoundariesFromSequence(seq):
  temp = list(seq)
  if temp[0] == leftEdge:
    temp = temp[1:]
  if temp[-1] == rightEdge:
    temp = temp[:-1]
  return tuple(temp)

#...and the number of sequences n is s.t. 2 ≤ n ≤ 5 
numInputs = sampleFrom(Uniform(range(2,6)))
print("{0} ≤ {1}".format("numInputs = |'lexicon'|",numInputs))
inputs = set([generateInputSequence(channelInput_i) for each in range(numInputs)])
if useEdgeSymbols:
  inputs = set(map(padInputSequenceWithBoundaries, inputs))
print("{0} = {1}".format("inputs = 'lexicon'",inputs))

# definition of p(X_0^f)
inputDist = Uniform(inputs)
print("p(X_0^f) = prior over the lexicon = uniform:")
inputDist

numInputs = |'lexicon'| ≤ 5
inputs = 'lexicon' = {('⋊', 1, 0, '⋉'), ('⋊', 0, 1, 1, 1, '⋉'), ('⋊', 1, 1, 0, 0, '⋉'), ('⋊', 1, '⋉')}
p(X_0^f) = prior over the lexicon = uniform:


ProbDist({('⋊', 0, 1, 1, 1, '⋉'): Fraction(1, 4),
          ('⋊', 1, '⋉'): Fraction(1, 4),
          ('⋊', 1, 0, '⋉'): Fraction(1, 4),
          ('⋊', 1, 1, 0, 0, '⋉'): Fraction(1, 4)})

As you can see above, we've defined $p(X_0^f)$ as a distribution object - we can reason about the whole distribution, calculate probabilities and conditional probabilities, and sample from the distribution/derived conditional distributions using the boilerplate code defined earlier.

### Convenient values/functions defiend in terms of $p(X_0^f)$ (prefixes, suffixes, etc.)

In [11]:
# convenient values and functions defined in terms of inputDist, part 1

import itertools
inputAlphabet = set(itertools.chain.from_iterable([set(k) for k in list(inputDist.keys())]))
print(inputAlphabet)
print(inputAlphabet - edgeSymbols)

{0, 1, '⋊', '⋉'}
{0, 1}


In [12]:
# convenient values and functions defined in terms of inputDist, part 2
def getPrefixes(s):
    return set(s[0:i] for i in range(1, len(s)+1))
test_input = list(inputs)[0]
print('TEST INPUT = {0}'.format(test_input))
print('Prefixes of {0}:'.format(test_input))
print(getPrefixes(test_input))

prefixes = map(getPrefixes, inputs)
prefixes = set(itertools.chain.from_iterable(prefixes))
print("<= 10 prefixes of lexicon:")
print(list(prefixes)[0:11])
def getHasPrefix(prefix):
    l = len(prefix)
    hasAsPrefix = lambda full_input_seq: full_input_seq[0:l] == prefix
    return hasAsPrefix
test_prefix = list(prefixes)[1]
print('TEST PREFIX = {0}'.format(test_prefix))
q = getHasPrefix(test_prefix)
for each in list(inputs)[0:11]:
    print("{0} has {1} as prefix?: {2}".format(each, test_prefix, q(each)))


TEST INPUT = ('⋊', 1, 0, '⋉')
Prefixes of ('⋊', 1, 0, '⋉'):
{('⋊',), ('⋊', 1, 0, '⋉'), ('⋊', 1, 0), ('⋊', 1)}
<= 10 prefixes of lexicon:
[('⋊', 1, 1), ('⋊', 1, 0), ('⋊', 1, 0, '⋉'), ('⋊', 0, 1, 1, 1), ('⋊', 1, 1, 0, 0), ('⋊', 0, 1, 1, 1, '⋉'), ('⋊', 0, 1, 1), ('⋊',), ('⋊', 1, 1, 0), ('⋊', 0, 1), ('⋊', 1, 1, 0, 0, '⋉')]
TEST PREFIX = ('⋊', 1, 0)
('⋊', 1, 0, '⋉') has ('⋊', 1, 0) as prefix?: True
('⋊', 0, 1, 1, 1, '⋉') has ('⋊', 1, 0) as prefix?: False
('⋊', 1, 1, 0, 0, '⋉') has ('⋊', 1, 0) as prefix?: False
('⋊', 1, '⋉') has ('⋊', 1, 0) as prefix?: False


In [13]:
# convenient values and functions defined in terms of inputDist 3
def getHasSuffix(suffix):
    l = len(suffix)
    hasAsSuffix = lambda full_input_seq: full_input_seq[-len(suffix):] == suffix
    return hasAsSuffix
test_suffix = test_input[len(test_input)-2:]
print('TEST SUFFIX: {0}'.format(test_suffix))
q = getHasSuffix(test_suffix)
for each in list(inputs)[0:11]:
    print("{0} has {1} as suffix?: {2}".format(each, test_suffix, q(each)))

TEST SUFFIX: (0, '⋉')
('⋊', 1, 0, '⋉') has (0, '⋉') as suffix?: True
('⋊', 0, 1, 1, 1, '⋉') has (0, '⋉') as suffix?: False
('⋊', 1, 1, 0, 0, '⋉') has (0, '⋉') as suffix?: True
('⋊', 1, '⋉') has (0, '⋉') as suffix?: False


As you can see from the 3-4 cells above, we can now
 - calculate the set of all prefixes of a sequence (and therefore of the 'lexicon')
 - check whether one sequence is a prefix of another sequence
 - check whether one sequence is a suffix of another sequence.

In [14]:
# convenient values and functions defined in terms of inputDist 3

# convenient calculation of p(x_0^i) as pmf, not as 'dist' 
def prefixProb(input_prefix):
    hasAsPrefix = getHasPrefix(input_prefix)
    return P(hasAsPrefix, inputDist)
print("p(X_0^f):")
print(inputDist)
print("TEST PREFIX: {0}".format(test_prefix))
print('p(x_0^i = {0}) = {1}'.format(test_prefix, prefixProb(test_prefix)))


# definition of p(X_0^f | x_0^i) as dist
def inputDist_givenPrefix(prefix):
    hasPrefix = getHasPrefix(prefix)
    return inputDist & hasPrefix
print('p(X_0^f| x_0^i = {0}):'.format(test_prefix))
print(inputDist_givenPrefix(test_prefix))

# definition of p(X_{i+1}^f| x_0^i) as pmf, not as 'dist'
def suffixProb(input_suffix, input_prefix):
    #p(X_0^f | x_0^i)
    d = inputDist_givenPrefix(input_prefix)
    isExactlyEqualTo = lambda s: s == tuple(list(input_prefix) + list(input_suffix))
    return P(isExactlyEqualTo, d)
#     hasAsSuffix = getHasSuffix(input_suffix)
#     return P(hasAsSuffix, d)
print('TEST SUFFIX: {0}'.format(test_suffix))
print('Let j = i+1:')
print(' p(x_j^f = {0}|x_0^i = {1}) = {2}'.format(test_suffix, test_prefix, suffixProb(test_suffix, test_prefix)))

p(X_0^f):
('⋊', 1, 0, '⋉'): 1/4 = 0.25
('⋊', 0, 1, 1, 1, '⋉'): 1/4 = 0.25
('⋊', 1, 1, 0, 0, '⋉'): 1/4 = 0.25
('⋊', 1, '⋉'): 1/4 = 0.25

TEST PREFIX: ('⋊', 1, 0)
p(x_0^i = ('⋊', 1, 0)) = 1/4
p(X_0^f| x_0^i = ('⋊', 1, 0)):
('⋊', 1, 0, '⋉'): 1/1 = 1.0

TEST SUFFIX: (0, '⋉')
Let j = i+1:
 p(x_j^f = (0, '⋉')|x_0^i = ('⋊', 1, 0)) = 0


We can now 
 - calculate $p(x_0^i)$
 - reason about the distribution $p(X_0^f|x_0^i)$
 - calculate $p(X_{i+1}^f| x_0^i)$.

## Defining $p(Y_j|x_j)$ and related channel pmfs

In [15]:
# definition of p(Y_i|X_i) as a dist

pError_i = round(random.uniform(0.1,0.4), 2)
pNoError_i = 1 - pError_i

channelOutput_i_dist = {
    0:         ProbDist({0: pNoError_i, 1:pError_i}),
    1:         ProbDist({0: pError_i, 1:pNoError_i}),
    leftEdge:  ProbDist({leftEdge: 1}),
    rightEdge: ProbDist({rightEdge: 1})
}

# def channelOutput_i_dist(inputSymbol_i):
#     if inputSymbol_i == 0:
#         return ProbDist({0: pNoError_i, 1:pError_i})
#     elif inputSymbol_i == 1:
#         return ProbDist({0: pError_i, 1:pNoError_i})
#     elif inputSymbol_i == leftEdge or inputSymbol_i == rightEdge:
#         return ProbDist({inputSymbol_i: 1})

print('p(Y_i|X_i = 0):')
# print(channelOutput_i_dist(0))
print(channelOutput_i_dist[0])
print('p(Y_i|X_i = 1):')
# print(channelOutput_i_dist(1))
print(channelOutput_i_dist[1])

p(Y_i|X_i = 0):
0: 0.75
1: 0.25

p(Y_i|X_i = 1):
0: 0.25
1: 0.75



We can now reason about the channel distribution at a single 'slice': $p(Y_j|x_j)$.

In [16]:
# definition of p(Y_i) as a dist
# channelOutput_i_marginal_dist = ProbDist({y: sum(channelOutput_i_dist(x)[y] for x in inputAlphabet) for y in inputAlphabet})
channelOutput_i_marginal_dist = ProbDist({y: sum(channelOutput_i_dist[x][y] for x in inputAlphabet) for y in inputAlphabet})
print("p(Y_i):")
print(channelOutput_i_marginal_dist)

p(Y_i):
0: 0.25
1: 0.25
'⋊': 0.25
'⋉': 0.25



...as well as the marginal distribution $p(Y_i)$. 

(Strictly speaking, the notation here is inaccurate/misleading with respect to the use of the index: 
 - for any given lexicon, the marginal distributions e.g. $p(Y_{i=0})$ and $p(Y_{i=1})$ can be different because $p(X_{i=0})$ and $p(X_{i=1})$ will in general be different
 - as well, word edge symbols have an a priori known index and are always perfectly transmitted -- i.e. $p(Y_i)$ depends on $i$, 

In spite of such facts, the function defined above doesn't make use of $i$-like information at all, but that's as intended -- it's supposed to represent the marginal distribution over $Y$ given some $x$ and no other information.)

Turning to the channel distribution (defined below)...

$p(y_0^i|\hat{x_0^i}) = p(y_0^i|x_0^i) = \prod_\limits{j=0}^{j=i} p(y_j|x_j)$, and where $p(y_j|x_j)$ doesn't depend on $j$ / is independent of $j$ given $x_j$, but what should $$p(y_0^j|x_0^i)$$ be calculated?

There are three cases:
 - $|y_0^j| = |x_0^i|$, where everything is simple: $p(y_0^i|x_0^i) = \prod_\limits{k=0}^{k=i} p(y_k|x_k)$
 - $|y_0^j| < |x_0^i|$, where we truncate irrelevant information (viz. the value of $x_{j+1}^i$) about the actually produced prefix: $p(y_0^j|x_0^i) = p(y_0^j|x_0^j) =  \prod_\limits{k=0}^{k=j} p(y_k|x_k)$
 - $|y_0^j| > |x_0^i|$, where we marginalize over future continuations of the actually produced prefix: $p(y_0^i|x_0^i) = \sum_\limits{x_{i+1}^j} p(x_{i+1}^j, y_0^j|x_0^i) = \sum_\limits{x_{i+1}^j} p(y_0^j|x_0^j)p(x_{i+1}^j|x_0^i) = \sum_\limits{\{x_0^j \, | \, x_0^i \text{ is a prefix}\}} p(y_0^j|x_0^j)\frac{p(x_0^j)}{p(x_0^i)}$

In [17]:
# definition of p(y_0^j|x_0^i) as a pmf but not as a 'dist', assuming uniphone noise/channel model
def channelOutput_prefix_prob(inputPrefix, outputPrefix):
    if len(outputPrefix) == len(inputPrefix):
    #         |y_0^j|    ==      |x_0^i|
#         slice_prob = lambda y_i, x_i: P({y_i}, channelOutput_i_dist(x_i))
        slice_prob = lambda y_i, x_i: P({y_i}, channelOutput_i_dist[x_i])
        return prod(slice_prob(y, x) for y,x in zip(outputPrefix, inputPrefix))
    elif len(outputPrefix) < len(inputPrefix): #truncate excess part of inputPrefix
              #|y_0^j|     <      |x_0^i|
        return channelOutput_prefix_prob(inputPrefix[:len(outputPrefix)], outputPrefix)
    else:
              #|y_0^j|     >      |x_0^i|
        hasInputPrefixAsPrefix = getHasPrefix(inputPrefix)
        my_prefixes = (pref for pref in prefixes if len(pref) == len(outputPrefix) and hasInputPrefixAsPrefix(pref))
        #         p(y_0^j|x_0^j)                                 p(x_0^j)          1   / p(x_0^i)
        terms = ((channelOutput_prefix_prob(pref, outputPrefix), prefixProb(pref), 1.0 / prefixProb(inputPrefix)) for pref in my_prefixes)
        return sum(map(prod, terms))

        

print("p(X_0^f):")
print(inputDist)
  
test_input_full = list(inputs)[0]
test_output_full = list(inputs)[0]
print("TEST INPUT full: {0}".format(test_input_full))
print("TEST OUTPUT full: {0}".format(test_output_full))
print('p(y_0^j = {0}| x_0^i = {1}) = {2}'.format(test_output_full, test_input_full, channelOutput_prefix_prob(test_input_full, test_output_full)))
print('len({0}) = {1}'.format(trimBoundariesFromSequence(test_input_full), len(trimBoundariesFromSequence(test_input_full))))
print('p(no_error_i) = {0}'.format(pNoError_i))
print('{2}^(len({0})) = {1}'.format(trimBoundariesFromSequence(test_input_full), pNoError_i ** len(trimBoundariesFromSequence(test_input_full)), pNoError_i))

print(' ')
test_input_slice = list(inputs)[0][0:2]
test_output_slice = list(inputs)[0][0:2]
print("TEST INPUT prefix: {0}".format(test_input_slice))
print("TEST OUTPUT prefix: {0}".format(test_output_slice))
print('p(y_0^j = {0}| x_0^i = {1}) = {2}'.format(test_output_slice, test_input_slice, channelOutput_prefix_prob(test_input_slice, test_output_slice)))

print(' ')
test_input_slice = list(inputs)[0][0:2]
test_output_slice = list(inputs)[0][0:3]
print("TEST INPUT prefix: {0}".format(test_input_slice))
print("TEST OUTPUT prefix: {0}".format(test_output_slice))
print('p(y_0^j = {0}| x_0^i = {1}) = {2}'.format(test_output_slice, test_input_slice, channelOutput_prefix_prob(test_input_slice, test_output_slice)))

print(' ')
test_input_slice = list(inputs)[0][0:3]
test_output_slice = list(inputs)[0][0:2]
print("TEST INPUT prefix: {0}".format(test_input_slice))
print("TEST OUTPUT prefix: {0}".format(test_output_slice))
print('p(y_0^j = {0}| x_0^i = {1}) = {2}'.format(test_output_slice, test_input_slice, channelOutput_prefix_prob(test_input_slice, test_output_slice)))

p(X_0^f):
('⋊', 1, 0, '⋉'): 1/4 = 0.25
('⋊', 0, 1, 1, 1, '⋉'): 1/4 = 0.25
('⋊', 1, 1, 0, 0, '⋉'): 1/4 = 0.25
('⋊', 1, '⋉'): 1/4 = 0.25

TEST INPUT full: ('⋊', 1, 0, '⋉')
TEST OUTPUT full: ('⋊', 1, 0, '⋉')
p(y_0^j = ('⋊', 1, 0, '⋉')| x_0^i = ('⋊', 1, 0, '⋉')) = 0.5625
len((1, 0)) = 2
p(no_error_i) = 0.75
0.75^(len((1, 0))) = 0.5625
 
TEST INPUT prefix: ('⋊', 1)
TEST OUTPUT prefix: ('⋊', 1)
p(y_0^j = ('⋊', 1)| x_0^i = ('⋊', 1)) = 0.75
 
TEST INPUT prefix: ('⋊', 1)
TEST OUTPUT prefix: ('⋊', 1, 0)
p(y_0^j = ('⋊', 1, 0)| x_0^i = ('⋊', 1)) = 0.25
 
TEST INPUT prefix: ('⋊', 1, 0)
TEST OUTPUT prefix: ('⋊', 1)
p(y_0^j = ('⋊', 1)| x_0^i = ('⋊', 1, 0)) = 0.75


We can now calculate $p(y_0^i|x_0^i)$...

In [18]:
# definition of sampler from p(Y_0^i|x_0^i), assuming uniphone noise/channel model
def channelOutput_prefix_sampler(inputPrefix):
#     return tuple([sampleFrom(channelOutput_i_dist(x_i)) for x_i in inputPrefix])
    return tuple([sampleFrom(channelOutput_i_dist[x_i]) for x_i in inputPrefix])

print("p(X_0^f):")
print(inputDist)

print("TEST INPUT prefix: {0}".format(test_input_slice))
print('10 samples from p(Y_0^i|x_0^i = {0}): {1}'.format(test_input_slice, list(sampleFrom(lambda : channelOutput_prefix_sampler(test_input_slice), 10)) ))

p(X_0^f):
('⋊', 1, 0, '⋉'): 1/4 = 0.25
('⋊', 0, 1, 1, 1, '⋉'): 1/4 = 0.25
('⋊', 1, 1, 0, 0, '⋉'): 1/4 = 0.25
('⋊', 1, '⋉'): 1/4 = 0.25

TEST INPUT prefix: ('⋊', 1, 0)
10 samples from p(Y_0^i|x_0^i = ('⋊', 1, 0)): [('⋊', 1, 0), ('⋊', 1, 0), ('⋊', 1, 0), ('⋊', 0, 0), ('⋊', 0, 0), ('⋊', 1, 0), ('⋊', 1, 0), ('⋊', 1, 0), ('⋊', 1, 0), ('⋊', 1, 1)]


...and sample from $p(Y_0^i|x_0^i)$.

In [19]:
# calculation of p(y_0^i) as a pmf but not as a 'dist'
#   marginalizes over all prefixes of input sequences with length matching y_0^i 
def channelOutput_prefix_marginal_prob(outputPrefix):

    l = len(outputPrefix)
    my_prefixes = (prefix for prefix in prefixes if len(prefix) == l)
        
    #\sum_{X_0^i} p(x_0^i) * p(y_0^i|x_0^i)
    terms = ((prefixProb(input_prefix), channelOutput_prefix_prob(input_prefix, outputPrefix)) for input_prefix in my_prefixes)
    probs = map(prod, terms)
#     probs = (prefixProb(input_prefix) * channelOutput_prefix_prob(input_prefix, outputPrefix) for input_prefix in my_prefixes)
#     probs = [prefixProb(input_prefix) * channelOutput_prefix_prob(input_prefix, outputPrefix) for input_prefix in my_prefixes] #only for testing; use generator exp otherwise
#     print('terms of \sum_{X_0^i} p(x_0^i) * p(y_0^i|x_0^i):')
#     print(probs)
    return sum(probs)

print("p(X_0^f):")
print(inputDist)

print('TEST OUTPUT prefix = {0}'.format(test_output_slice))
print('p(y_0^i = {0}) = {1}'.format(test_output_slice, channelOutput_prefix_marginal_prob(test_output_slice)))

p(X_0^f):
('⋊', 1, 0, '⋉'): 1/4 = 0.25
('⋊', 0, 1, 1, 1, '⋉'): 1/4 = 0.25
('⋊', 1, 1, 0, 0, '⋉'): 1/4 = 0.25
('⋊', 1, '⋉'): 1/4 = 0.25

TEST OUTPUT prefix = ('⋊', 1)
p(y_0^i = ('⋊', 1)) = 0.625


We can now calculate $p(y_0^i)$.

## Defining $\hat{p}(\widehat{x}_0^j|x_0^i)$

$p(\hat{x_0^j}|x_0^i)$ is a more general object to calculate than $p(\hat{x_0^f}|x_0^i)$. 

$p(\hat{x_0^j}|x_0^i)$ has two cases: $|\hat{x}_0^j| \geq |x_0^i|$ and $|\hat{x}_0^j| < |x_0^i|$.

Assuming $|\hat{x}_0^j| \geq |x_0^i|$:
$$p(\hat{x_0^j}|x_0^i) = \sum_\limits{y_0^i} p(\hat{x_0^j}|y_0^i)p(y_0^i|x_0^i) = \sum_\limits{y_0^i} \frac{p(y_0^i|\hat{x_0^j})p(\hat{x_0^j})}{p(y_0^i)}p(y_0^i|x_0^i) = \sum_\limits{y_0^i} \frac{p(y_0^i|\hat{x_0^i})p(\hat{x_0^j})}{p(y_0^i)}p(y_0^i|x_0^i) $$

Assuming $|\hat{x}_0^j| < |x_0^i|$:
$$p(\hat{x_0^j}|x_0^i) = \sum_\limits{\hat{x_{j+1}^i}} p(\hat{x_0^j}, \hat{x_{j+1}^i}|x_0^i) = \sum_\limits{\hat{x_{j+1}^i}} p(\hat{x_0^i}|x_0^i) = \sum_\limits{\{\hat{x_0^i} \, | \, \hat{x_0^j} \text{ is a prefix}\}} p(\hat{x_0^i}|x_0^i)$$

In [20]:
# mc estimate of p(x-hat_0^j|x_0^i) as pmf, not as 'dist', assuming uniphone error probs
#                         x_0^i             x-hat_0^j
def est_channelInput_prob(true_inputPrefix, poss_inputPrefix, num_samples = None):
    if num_samples == None:
        num_samples = 1000
    
#     print('true x_0^i: {0}'.format(true_inputPrefix))
#     print('x-hat_0^j: {0}'.format(poss_inputPrefix))
#     print('num samples: {0}'.format(num_samples))
    
    lenTruePrefix = len(true_inputPrefix) #|x_0^i|
    lenPossPrefix = len(poss_inputPrefix) #|x-hat_0^j|

    #if |x-hat_0^j|  < |x_0^i|, then calculate \sum_\limits{x-hat_0^i | x-hat_0^j is a prefix} p(x-hat_0^i|x_0^i)
    if lenPossPrefix < lenTruePrefix:
        hasPossPrefixAsPrefix = getHasPrefix(poss_inputPrefix)
        my_prefixes = (pref for pref in prefixes if len(pref) == lenTruePrefix and hasPossPrefixAsPrefix(pref))
        probs = (est_channelInput_prob(true_inputPrefix, pref) for pref in my_prefixes)
        return sum(probs)

    #if |x-hat_0^j| ≥ |x_0^i|, then proceed 'normally'...
    
    #samples from p(Y_0^i|x_0^i)
    samples = sampleFrom(lambda: channelOutput_prefix_sampler(true_inputPrefix), num_samples)
#     samples = list(sampleFrom(lambda: channelOutput_prefix_sampler(true_inputPrefix), num_samples)) # only for debugging - otherwise use generator exp
#     print('samples from p(Y_0^i|x_0^i):')
#     print(samples)
#     likelihoods = [channelOutput_prefix_prob(poss_inputPrefix, outputPrefix) for outputPrefix in samples]
#     print('likelihoods p(y_0^i|x-hat_0^i):')
#     print(likelihoods)
#     priorOfOutputs = [channelOutput_prefix_marginal_prob(outputPrefix) for outputPrefix in samples]
#     print('priorOfOutputs p(y_0^i):')
#     print(priorOfOutputs)
    
    #p(x-hat_0^j)
    poss_inputPrefix_prob = prefixProb(poss_inputPrefix)
#     print('p(x-hat_0^j) = {0}'.format( poss_inputPrefix_prob ))
    
    #terms in \sum_{samples y_0^i from p(Y_0^i|x_0^i)} p(y_0^i|x-hat_0^i) * p(x-hat_0^j) / p(y_0^i)
    terms = (channelOutput_prefix_prob(poss_inputPrefix, outputPrefix)  * poss_inputPrefix_prob / channelOutput_prefix_marginal_prob(outputPrefix) for outputPrefix in samples)
#     for outputPrefix in samples:
#       print('y_0^i: {0}'.format(outputPrefix))
#       print('term: {0} * {1} / {2}'.format(channelOutput_prefix_prob(poss_inputPrefix, outputPrefix), poss_inputPrefix_prob, channelOutput_prefix_marginal_prob(outputPrefix)))
#     terms = [channelOutput_prefix_prob(poss_inputPrefix, outputPrefix)  * poss_inputPrefix_prob / channelOutput_prefix_marginal_prob(outputPrefix) for outputPrefix in samples] #use only for debugging, otherwise use generator exp
#     print('terms in sum:')
#     print(terms)
    est = sum(terms) / num_samples
    
    return est

print("p(X_0^f):")
print(inputDist)

print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(('⋊', 1), test_input_slice, est_channelInput_prob(test_input_slice, ('⋊', 1))))
print(' ')

print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(test_input_slice, test_input_slice, est_channelInput_prob(test_input_slice, test_input_slice)))
print(' ')
print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(test_input_full, test_input_slice, est_channelInput_prob(test_input_slice, test_input_full)))
print(' ')
print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(test_input_slice, test_input_full, est_channelInput_prob(test_input_full, test_input_slice)))

p(X_0^f):
('⋊', 1, 0, '⋉'): 1/4 = 0.25
('⋊', 0, 1, 1, 1, '⋉'): 1/4 = 0.25
('⋊', 1, 1, 0, 0, '⋉'): 1/4 = 0.25
('⋊', 1, '⋉'): 1/4 = 0.25

est p(x-hat_0^j = ('⋊', 1)| x_0^i = ('⋊', 1, 0)) = 0.7974505494505422
 
est p(x-hat_0^j = ('⋊', 1, 0)| x_0^i = ('⋊', 1, 0)) = 0.5186109890109841
 
est p(x-hat_0^j = ('⋊', 1, 0, '⋉')| x_0^i = ('⋊', 1, 0)) = 0.49687912087911534
 
est p(x-hat_0^j = ('⋊', 1, 0)| x_0^i = ('⋊', 1, 0, '⋉')) = 1.0


# Generating test lexicons and noise distributions

## Generating $p(X_0^f)$

To generate a codebook = set of channel input sequences = lexicon of bitstrings, we need to decide
 - how big or small lexicons can be
 - how big or small words can be
 - what the marginal distribution over symbols is within a word.

In [24]:
# Boilerplate...

# Symbols

bits = set([0, 1])

# p(X_i)
inputDist_i = Uniform(bits)
print('Distribution over symbols at each index of a word:')
print(inputDist_i)

#min, max length of a channel input sequence (codeword = wordform)
min_input_seq_length = 1
max_input_seq_length = 5
print('Length of input sequences = codewords = wordforms should be between {0} and {1} (inclusive).'.format(min_input_seq_length, max_input_seq_length))

#min, max number of channel input sequences (= codewords = wordforms)
min_inputs = 2
max_inputs = 5
print('Size of lexicon = # of possible channel inputs should be between {0} and {1} (inclusive).'.format(min_inputs, max_inputs))

#define element of support of p(X_0^f), where min_num_symbols_per_word - 1 ≤ f ≤ max_num_symbols_per_word...
def generateInputSequence(x_i_sampler, min_length, max_length):
    length = sampleFrom(Uniform(range(min_length, max_length)))
    return tuple(x_i_sampler() for each in range(length))

def padInputSequenceWithBoundaries(inputSeq):
    temp = list(inputSeq)
    temp = tuple([leftEdge] + temp + [rightEdge])
    return temp

def trimBoundariesFromSequence(seq):
    temp = list(seq)
    if temp[0] == leftEdge:
        temp = temp[1:]
    if temp[-1] == rightEdge:
        temp = temp[:-1]
    return tuple(temp)


def generate_input_dist(min_num_words, max_num_words, 
                        min_num_symbols_per_word, max_num_symbols_per_word,
                        input_dist_i,
                        useEdgeSymbols = None):
    if useEdgeSymbols == None:
        useEdgeSymbols = True
    
    # function that samples from p(X_i)
    channelInput_i = makeSampler(inputDist_i)
    
    
    numInputs = sampleFrom(Uniform(range(min_num_words, max_num_words)))
#     print("{0} ≤ {1}".format("numInputs = |'lexicon'|",numInputs))
    
    # the possible of values X_0^f
    inputs = set([generateInputSequence(channelInput_i, min_num_symbols_per_word, max_num_symbols_per_word) for each in range(numInputs)])
    if useEdgeSymbols:
        inputs = set(map(padInputSequenceWithBoundaries, inputs))
#     print("{0} = {1}".format("inputs = 'lexicon'",inputs))

    if len(inputs) < min_num_words:
        return generate_input_dist(min_num_words, max_num_words, 
                        min_num_symbols_per_word, max_num_symbols_per_word,
                        input_dist_i, useEdgeSymbols)    
    
    # assume uniform dist over inputs
    dist = Uniform(inputs)
    
    my_inputAlphabet = union(map(set,dist.keys()))
    alphabet_minus_edges = set([each for each in my_inputAlphabet if each not in edgeSymbols])
    if len(alphabet_minus_edges) == 1:
        return generate_input_dist(min_num_words, max_num_words, 
                        min_num_symbols_per_word, max_num_symbols_per_word,
                        input_dist_i, useEdgeSymbols)   
    
    return dist

Distribution over symbols at each index of a word:
0: 1/2 = 0.5
1: 1/2 = 0.5

Length of input sequences = codewords = wordforms should be between 1 and 5 (inclusive).
Size of lexicon = # of possible channel inputs should be between 2 and 5 (inclusive).


In [25]:
print(' ')
print('Randomly generated input distribution:')
inputDist = generate_input_dist(min_inputs, max_inputs, min_input_seq_length, max_input_seq_length, True)
inputDist

 
Randomly generated input distribution:


ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
          ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)})

## Generating $p(Y_i|x_i)$

In [26]:
# definition of p(Y_i|X_i) as a conditional distribution

def generate_error_probability():
    return round(random.uniform(0.1,0.4), 2)

def generate_channel_dist_i(all_channel_input_symbols, error_prob = None):
    assert len(all_channel_input_symbols) > 1, "There must be >1 channel input symbol in {0}.".format(all_channel_input_symbols)

    #Edge symbols are assumed to be perfectly transmitted and no non-edge symbol is ever confused with a word edge.
    inputAlphabet_withoutEdges = [each for each in all_channel_input_symbols if each not in edgeSymbols]
    
    if error_prob == None:
        pError_i = generate_error_probability()
    else:
        assert error_prob >= 0.0 and error_prob <= 1.0, 'Error probability {0} must be >= 0 and <= 1.0!'.format(error_prob)
    pNoError_i = 1 - pError_i
    pEachError_i = pError_i / (len(inputAlphabet_withoutEdges) - 1)
#     print('P(error) = {0}'.format(pError_i))

    
    getProb = lambda key_x, other_x: pNoError_i if key_x == other_x else pEachError_i
    
    #constructs p(Y_i|x_i) for a particular x_i where 
    #  p(y_i|x_i) = pNoError_i iff y_i = x_i
    #  p(y_i|x_i) = 1 / n, where n = the number of symbols y_i != x_i
    getConditionalChannelDist = lambda key_x: ProbDist({x:getProb(key_x,x) for x in inputAlphabet_withoutEdges})

    channel_dist_i = {x:getConditionalChannelDist(x) for x in inputAlphabet_withoutEdges}
    
    #Now we fix left edge and right edge symbols so that they are always perfectly transmitted:
    channel_dist_i[leftEdge] = ProbDist({leftEdge: 1})
    channel_dist_i[rightEdge] = ProbDist({rightEdge: 1})
    
    return channel_dist_i


# channelOutput_i_dist = {
#     0:         ProbDist({0: pNoError_i, 1:pError_i}),
#     1:         ProbDist({0: pError_i, 1:pNoError_i}),
#     leftEdge:  ProbDist({leftEdge: 1}),
#     rightEdge: ProbDist({rightEdge: 1})
# }
print('Channel output distribution at any given index:')
inputAlphabet = union(map(set,inputDist.keys()))
channelOutput_i_dist = generate_channel_dist_i( inputAlphabet )
channelOutput_i_dist

Channel output distribution at any given index:


{0: ProbDist({0: 0.85, 1: 0.15}),
 1: ProbDist({0: 0.15, 1: 0.85}),
 '⋉': ProbDist({'⋉': Fraction(1, 1)}),
 '⋊': ProbDist({'⋊': Fraction(1, 1)})}

# Generating model code

## Input/source model...

In [28]:
# Generating functions and values derived from p(X_0^f)...

import itertools

def getPrefixes(s):
    return set(s[0:i] for i in range(1, len(s)+1))

def getHasPrefix(prefix):
    l = len(prefix)
    hasAsPrefix = lambda full_input_seq: full_input_seq[0:l] == prefix
    return hasAsPrefix
  
def getHasSuffix(suffix):
    l = len(suffix)
    hasAsSuffix = lambda full_input_seq: full_input_seq[-len(suffix):] == suffix
    return hasAsSuffix

def padInputSequenceWithBoundaries(inputSeq):
    temp = list(inputSeq)
    temp = tuple([leftEdge] + temp + [rightEdge])
    return temp

def trimBoundariesFromSequence(seq):
    temp = list(seq)
    if temp[0] == leftEdge:
        temp = temp[1:]
    if temp[-1] == rightEdge:
        temp = temp[:-1]
    return tuple(temp)

def generateInputModel(inputDist, pad_with_boundaries = None):
    if pad_with_boundaries == None:
        pad_with_boundaries = False
    model = {} #FIXME consider changing to a namedtuple/frozendict (possibly just before returning a value)
    
    if pad_with_boundaries:
        old_map = list(inputDist.items())
        old_inputs = list(map(lambda p: p[0], old_map))
#         old_inputs = list(inputDist.keys())
        old_values = [v for k,v in old_map]
        new_inputs = list(map(padInputSequenceWithBoundaries, old_inputs))
        assert(list(map(trimBoundariesFromSequence, new_inputs)) == old_inputs)
        assert(list(zip(old_inputs, old_values)) == old_map)
        new_map = dict(list(zip(new_inputs, old_values)))
        inputDist = new_map
    
    model['inputDist'] = inputDist
    model['p(X_0^f)'] = inputDist
    
    inputs = list(inputDist.keys())
    model['inputs'] = inputs
    
    
    inputAlphabet = set(itertools.chain.from_iterable([set(k) for k in inputs ]))
#   print(inputAlphabet)
#   print(inputAlphabet - edgeSymbols)
#     if pad_with_boundaries and (leftEdge not in inputAlphabet):
#         inputAlphabet += [leftEdge]
#     if pad_with_boundaries and (rightEdge not in inputAlphabet):
#         inputAlphabet += [rightEdge]
    model['inputAlphabet'] = inputAlphabet
    

    prefixes = map(getPrefixes, inputs)
    prefixes = set(itertools.chain.from_iterable(prefixes))
#   print("<= 10 prefixes of lexicon:")
#   print(list(prefixes)[0:11])
    model['prefixes'] = prefixes
  
    
    # convenient calculation of p(x_0^i) as pmf, not as 'dist'
    def prefixProb(input_prefix):
        hasAsPrefix = getHasPrefix(input_prefix)
        return P(hasAsPrefix, inputDist)
    model['prefixProb'] = prefixProb
    model['p(x_0^i)'] = prefixProb
    
    
    # definition of p(X_0^f | x_0^i) as dist
    def inputDist_givenPrefix(prefix):
        hasPrefix = getHasPrefix(prefix)
        return inputDist & hasPrefix
    model['inputDist_givenPrefix'] = inputDist_givenPrefix
    model['p(X_0^f | x_0^i)'] = inputDist_givenPrefix
    
    
    # definition of p(x_{i+1}^f | x_0^i) as pmf, not as 'dist'
    def suffixProb(input_suffix, input_prefix):
        #p(X_0^f | x_0^i)
        d = inputDist_givenPrefix(input_prefix)
        isExactlyEqualTo = lambda s: s == tuple(list(input_prefix) + list(input_suffix))
        return P(isExactlyEqualTo, d)
    model['suffixProb'] = suffixProb
    model['p(x_{i+1}^f | x_0^i)'] = suffixProb
    
    return model

In [29]:
print('p(X_0^f):')
print(inputDist)
print('Derived values/functions...')
generateInputModel(inputDist, pad_with_boundaries = False)

p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

Derived values/functions...


{'inputAlphabet': {0, 1, '⋉', '⋊'},
 'inputDist': ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
           ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)}),
 'inputDist_givenPrefix': <function __main__.generateInputModel.<locals>.inputDist_givenPrefix>,
 'inputs': [('⋊', 0, 0, 1, '⋉'), ('⋊', 0, 0, '⋉')],
 'p(X_0^f | x_0^i)': <function __main__.generateInputModel.<locals>.inputDist_givenPrefix>,
 'p(X_0^f)': ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
           ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)}),
 'p(x_0^i)': <function __main__.generateInputModel.<locals>.prefixProb>,
 'p(x_{i+1}^f | x_0^i)': <function __main__.generateInputModel.<locals>.suffixProb>,
 'prefixProb': <function __main__.generateInputModel.<locals>.prefixProb>,
 'prefixes': {('⋊', 0),
  ('⋊', 0, 0),
  ('⋊', 0, 0, '⋉'),
  ('⋊', 0, 0, 1),
  ('⋊', 0, 0, 1, '⋉'),
  ('⋊',)},
 'suffixProb': <function __main__.generateInputModel.<locals>.suffixProb>}

## Channel model...

In [30]:
# Generating functions and values derived from p(Y_i|x_i)...


def generateChannelModel(channelOutput_i_dist, inputModel):
    
    model = {} #FIXME consider changing to a namedtuple/frozendict (possibly just before returning a value)
    
    model['channelOutput_i_dist'] = channelOutput_i_dist
    model['p(Y_i|x_i)'] = channelOutput_i_dist
    
    
    # definition of p(Y_i) as a dist
    sourceInputAlphabet = inputModel['inputAlphabet']
    
    
    getKeys = lambda d: set(d.keys()) #assumes they're hashable
#     getVals = lambda d: set(d.values()) #assumes they're hashable
    channelInputAlphabet = getKeys(channelOutput_i_dist)
    
    getOutputSymbols = lambda cond_channel_dist: getKeys(cond_channel_dist)
    outputSymbolsByInput = map(lambda eachInputSymbol: getOutputSymbols(channelOutput_i_dist[eachInputSymbol]), channelOutput_i_dist)
    channelOutputAlphabet = reduce(set.union, outputSymbolsByInput)
    
    totalAlphabet = reduce(set.union, [channelOutputAlphabet, channelInputAlphabet, sourceInputAlphabet])
    
    
#     channelOutput_i_marginal_dist = ProbDist({y: sum(channelOutput_i_dist(x)[y] for x in inputAlphabet) for y in inputAlphabet})
    channelOutput_i_marginal_dist = ProbDist({y: sum(channelOutput_i_dist[x].get(y, 0.0) for x in channelInputAlphabet) for y in channelOutputAlphabet})
    model['channelOutput_i_marginal_dist'] = channelOutput_i_marginal_dist
    model['p(Y_i)'] = channelOutput_i_marginal_dist
    
    
    # definition of p(y_0^j|x_0^i) as a pmf but not as a 'dist', assuming uniphone noise/channel model
    prefixes = inputModel['prefixes']
    def channelOutput_prefix_prob(inputPrefix, outputPrefix):
        if len(outputPrefix) == len(inputPrefix):
        #         |y_0^j|    ==      |x_0^i|
#             slice_prob = lambda y_i, x_i: P({y_i}, channelOutput_i_dist(x_i))
            slice_prob = lambda y_i, x_i: P({y_i}, channelOutput_i_dist[x_i])
            return prod(slice_prob(y, x) for y,x in zip(outputPrefix, inputPrefix))
        elif len(outputPrefix) < len(inputPrefix): #truncate excess part of inputPrefix
                  #|y_0^j|     <      |x_0^i|
            return channelOutput_prefix_prob(inputPrefix[:len(outputPrefix)], outputPrefix)
        else:
                  #|y_0^j|     >      |x_0^i|
            hasInputPrefixAsPrefix = getHasPrefix(inputPrefix)
            my_prefixes = (pref for pref in prefixes if len(pref) == len(outputPrefix) and hasInputPrefixAsPrefix(pref))
            #         p(y_0^j|x_0^j)                                 p(x_0^j)          1   / p(x_0^i)
            terms = ((channelOutput_prefix_prob(pref, outputPrefix), inputModel['prefixProb'](pref), 0.0 if inputModel['prefixProb'](inputPrefix) == 0.0 else 1.0 / inputModel['prefixProb'](inputPrefix) ) for pref in my_prefixes)
            return sum(map(prod, terms))
    model['p(y_0^j|x_0^i)'] = channelOutput_prefix_prob
    model['channelOutput_prefix_prob'] = channelOutput_prefix_prob


    # definition of sampler from p(Y_0^i|x_0^i), assuming uniphone noise/channel model
    def channelOutput_prefix_sampler(inputPrefix):
#         return tuple([sampleFrom(channelOutput_i_dist(x_i)) for x_i in inputPrefix])
        return tuple([sampleFrom(channelOutput_i_dist[x_i]) for x_i in inputPrefix])
    model['sample from p(Y_0^i|x_0^i)'] = channelOutput_prefix_sampler
    model['channelOutput_prefix_sampler'] = channelOutput_prefix_sampler

    
    # calculation of p(y_0^i) as a pmf but not as a 'dist'
    #   marginalizes over all prefixes of input sequences with length matching y_0^i 
    def channelOutput_prefix_marginal_prob(outputPrefix):

        l = len(outputPrefix)
        my_prefixes = (prefix for prefix in prefixes if len(prefix) == l)

        #\sum_{X_0^i} p(x_0^i) * p(y_0^i|x_0^i)
        terms = ((inputModel['prefixProb'](input_prefix), channelOutput_prefix_prob(input_prefix, outputPrefix)) for input_prefix in my_prefixes)
        probs = map(prod, terms)
    #     probs = (prefixProb(input_prefix) * channelOutput_prefix_prob(input_prefix, outputPrefix) for input_prefix in my_prefixes)
    #     probs = [prefixProb(input_prefix) * channelOutput_prefix_prob(input_prefix, outputPrefix) for input_prefix in my_prefixes] #only for testing; use generator exp otherwise
    #     print('terms of \sum_{X_0^i} p(x_0^i) * p(y_0^i|x_0^i):')
    #     print(probs)
        return sum(probs)
    model['p(y_0^i)'] = channelOutput_prefix_marginal_prob
    model['channelOutput_prefix_marginal_prob'] = channelOutput_prefix_marginal_prob
    
    
    
    return model

In [31]:
print('p(X_0^f):')
print(inputDist)

print('p(Y_i|X_i = 0):')
# print(channelOutput_i_dist(0))
print(channelOutput_i_dist[0])
print('p(Y_i|X_i = 1):')
# print(channelOutput_i_dist(1))
print(channelOutput_i_dist[1])

print('Derived values/functions...')
myInputModel = generateInputModel(inputDist)
myInputModel

p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

p(Y_i|X_i = 0):
0: 0.85
1: 0.15

p(Y_i|X_i = 1):
0: 0.15
1: 0.85

Derived values/functions...


{'inputAlphabet': {0, 1, '⋉', '⋊'},
 'inputDist': ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
           ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)}),
 'inputDist_givenPrefix': <function __main__.generateInputModel.<locals>.inputDist_givenPrefix>,
 'inputs': [('⋊', 0, 0, 1, '⋉'), ('⋊', 0, 0, '⋉')],
 'p(X_0^f | x_0^i)': <function __main__.generateInputModel.<locals>.inputDist_givenPrefix>,
 'p(X_0^f)': ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
           ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)}),
 'p(x_0^i)': <function __main__.generateInputModel.<locals>.prefixProb>,
 'p(x_{i+1}^f | x_0^i)': <function __main__.generateInputModel.<locals>.suffixProb>,
 'prefixProb': <function __main__.generateInputModel.<locals>.prefixProb>,
 'prefixes': {('⋊', 0),
  ('⋊', 0, 0),
  ('⋊', 0, 0, '⋉'),
  ('⋊', 0, 0, 1),
  ('⋊', 0, 0, 1, '⋉'),
  ('⋊',)},
 'suffixProb': <function __main__.generateInputModel.<locals>.suffixProb>}

In [32]:
myChannelModel = generateChannelModel(channelOutput_i_dist, myInputModel)
myChannelModel

{'channelOutput_i_dist': {0: ProbDist({0: 0.85, 1: 0.15}),
  1: ProbDist({0: 0.15, 1: 0.85}),
  '⋉': ProbDist({'⋉': Fraction(1, 1)}),
  '⋊': ProbDist({'⋊': Fraction(1, 1)})},
 'channelOutput_i_marginal_dist': ProbDist({0: 0.25,
           1: 0.25,
           '⋉': 0.25,
           '⋊': 0.25}),
 'channelOutput_prefix_marginal_prob': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_marginal_prob>,
 'channelOutput_prefix_prob': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_prob>,
 'channelOutput_prefix_sampler': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_sampler>,
 'p(Y_i)': ProbDist({0: 0.25, 1: 0.25, '⋉': 0.25, '⋊': 0.25}),
 'p(Y_i|x_i)': {0: ProbDist({0: 0.85, 1: 0.15}),
  1: ProbDist({0: 0.15, 1: 0.85}),
  '⋉': ProbDist({'⋉': Fraction(1, 1)}),
  '⋊': ProbDist({'⋊': Fraction(1, 1)})},
 'p(y_0^i)': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_marginal_prob>,
 'p(y_0^j|x_0^i)': <function __mai

## Total model...

In [33]:
from math import exp

#Hoeffding's inequality w.r.t. mc estimate p-hat(x-hat_0^j|x_0^i):
# Let  X = p-hat(x-hat_0^j|x_0^i)
#      𝛍 = p(x-hat_0^j|x_0^i)
#      n = the number of samples that go into calculating X
# then
#      p(|X - 𝛍| > 𝛆) ≤ exp(2n𝛆²)
#
#  E.g. for n = 1000, 𝛆 = 0.01
#      p(|X - 𝛍| > 𝛆) ≤ ≈0.82
#
#       for n = 1000, 𝛆 = 0.03
#     p(|X - 𝛍| > 𝛆) ≤ ≈0.165
#
#       for n = 1000, 𝛆 = 0.05
#     p(|X - 𝛍| > 𝛆) ≤ ≈0.0068
#
#       for n = 1000, 𝛆 = 0.1
#     p(|X - 𝛍| > 𝛆) ≤ ≈2.06*10^-9
def hoeffdingProb(epsilon, num_samples):
    upperBound = exp(-2.0 * num_samples * epsilon * epsilon)
    return upperBound

def generateTotalModel(inputDist, channelOutput_i_dist, pad_with_boundaries = None):
    if pad_with_boundaries == None:
        pad_with_boundaries = False
    
    inputModel = generateInputModel(inputDist, pad_with_boundaries = pad_with_boundaries)
    channelModel = generateChannelModel(channelOutput_i_dist, inputModel)
    model = {**inputModel, **channelModel} #FIXME consider changing to a namedtuple/frozendict (possibly just before returning a value)

    # receiver/listener model
    #  = mc estimate of p(x-hat_0^j|x_0^i) as pmf, not as 'dist', assuming uniphone error probs
    #                         x_0^i             x-hat_0^j
    def est_channelInput_prob(true_inputPrefix, poss_inputPrefix, num_samples = None):
        if num_samples == None:
            num_samples = 1000

    #     print('true x_0^i: {0}'.format(true_inputPrefix))
    #     print('x-hat_0^j: {0}'.format(poss_inputPrefix))
    #     print('num samples: {0}'.format(num_samples))

        lenTruePrefix = len(true_inputPrefix) #|x_0^i|
        lenPossPrefix = len(poss_inputPrefix) #|x-hat_0^j|

        #if |x-hat_0^j|  < |x_0^i|, then calculate \sum_\limits{x-hat_0^i | x-hat_0^j is a prefix} p(x-hat_0^i|x_0^i)
        if lenPossPrefix < lenTruePrefix:
            hasPossPrefixAsPrefix = getHasPrefix(poss_inputPrefix)
            my_prefixes = (pref for pref in model['prefixes'] if len(pref) == lenTruePrefix and hasPossPrefixAsPrefix(pref))
            probs = (est_channelInput_prob(true_inputPrefix, pref) for pref in my_prefixes)
            return sum(probs)

        #if |x-hat_0^j| ≥ |x_0^i|, then proceed 'normally'...

        #samples from p(Y_0^i|x_0^i)
        samples = sampleFrom(lambda: model['channelOutput_prefix_sampler'](true_inputPrefix), num_samples)
    #     samples = list(sampleFrom(lambda: channelOutput_prefix_sampler(true_inputPrefix), num_samples)) # only for debugging - otherwise use generator exp
    #     print('samples from p(Y_0^i|x_0^i):')
    #     print(samples)
    #     likelihoods = [channelOutput_prefix_prob(poss_inputPrefix, outputPrefix) for outputPrefix in samples]
    #     print('likelihoods p(y_0^i|x-hat_0^i):')
    #     print(likelihoods)
    #     priorOfOutputs = [channelOutput_prefix_marginal_prob(outputPrefix) for outputPrefix in samples]
    #     print('priorOfOutputs p(y_0^i):')
    #     print(priorOfOutputs)

        #p(x-hat_0^j)
        poss_inputPrefix_prob = model['prefixProb'](poss_inputPrefix)
    #     print('p(x-hat_0^j) = {0}'.format( poss_inputPrefix_prob ))

        #terms in \sum_{samples y_0^i from p(Y_0^i|x_0^i)} p(y_0^i|x-hat_0^i) * p(x-hat_0^j) / p(y_0^i)
#         terms = (model['channelOutput_prefix_prob'](poss_inputPrefix, outputPrefix)  * poss_inputPrefix_prob / model['channelOutput_prefix_marginal_prob'](outputPrefix) for outputPrefix in samples)
        terms = ((model['channelOutput_prefix_prob'](poss_inputPrefix, outputPrefix), 0.0 if model['channelOutput_prefix_marginal_prob'](outputPrefix) == 0.0 else poss_inputPrefix_prob / model['channelOutput_prefix_marginal_prob'](outputPrefix) ) for outputPrefix in samples)
    #     for outputPrefix in samples:
    #       print('y_0^i: {0}'.format(outputPrefix))
    #       print('term: {0} * {1} / {2}'.format(channelOutput_prefix_prob(poss_inputPrefix, outputPrefix), poss_inputPrefix_prob, channelOutput_prefix_marginal_prob(outputPrefix)))
    #     terms = [channelOutput_prefix_prob(poss_inputPrefix, outputPrefix)  * poss_inputPrefix_prob / channelOutput_prefix_marginal_prob(outputPrefix) for outputPrefix in samples] #use only for debugging, otherwise use generator exp
    #     print('terms in sum:')
    #     print(terms)
#         est = sum(terms) / num_samples
        est = sum(map(prod,terms)) / num_samples

        return est
    model['p-hat(x-hat_0^j|x_0^i)'] = est_channelInput_prob
    model['est_channelInput_prob'] = est_channelInput_prob
    
    return model

In [34]:
generateTotalModel(inputDist, channelOutput_i_dist)

{'channelOutput_i_dist': {0: ProbDist({0: 0.85, 1: 0.15}),
  1: ProbDist({0: 0.15, 1: 0.85}),
  '⋉': ProbDist({'⋉': Fraction(1, 1)}),
  '⋊': ProbDist({'⋊': Fraction(1, 1)})},
 'channelOutput_i_marginal_dist': ProbDist({0: 0.25,
           1: 0.25,
           '⋉': 0.25,
           '⋊': 0.25}),
 'channelOutput_prefix_marginal_prob': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_marginal_prob>,
 'channelOutput_prefix_prob': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_prob>,
 'channelOutput_prefix_sampler': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_sampler>,
 'est_channelInput_prob': <function __main__.generateTotalModel.<locals>.est_channelInput_prob>,
 'inputAlphabet': {0, 1, '⋉', '⋊'},
 'inputDist': ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
           ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)}),
 'inputDist_givenPrefix': <function __main__.generateInputModel.<locals>.inputDist_givenPrefix>,
 'inputs': [('⋊', 0, 0,

## Testing the generated code on the generated lexicon and noise distributions

The model inputs are restated below for convenience (and typically at the top of each cell...)

In [35]:
print('MODEL INPUTS:')

print('p(X_0^f):')
print(inputDist)

print('p(Y_i|X_i = 0):')
# print(channelOutput_i_dist(0))
print(channelOutput_i_dist[0])
print('p(Y_i|X_i = 1):')
# print(channelOutput_i_dist(1))
print(channelOutput_i_dist[1])

MODEL INPUTS:
p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

p(Y_i|X_i = 0):
0: 0.85
1: 0.15

p(Y_i|X_i = 1):
0: 0.15
1: 0.85



In [36]:
print('DERIVED VALUES/FUNCTIONS:')

total_model = generateTotalModel(inputDist, channelOutput_i_dist)
total_model

DERIVED VALUES/FUNCTIONS:


{'channelOutput_i_dist': {0: ProbDist({0: 0.85, 1: 0.15}),
  1: ProbDist({0: 0.15, 1: 0.85}),
  '⋉': ProbDist({'⋉': Fraction(1, 1)}),
  '⋊': ProbDist({'⋊': Fraction(1, 1)})},
 'channelOutput_i_marginal_dist': ProbDist({0: 0.25,
           1: 0.25,
           '⋉': 0.25,
           '⋊': 0.25}),
 'channelOutput_prefix_marginal_prob': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_marginal_prob>,
 'channelOutput_prefix_prob': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_prob>,
 'channelOutput_prefix_sampler': <function __main__.generateChannelModel.<locals>.channelOutput_prefix_sampler>,
 'est_channelInput_prob': <function __main__.generateTotalModel.<locals>.est_channelInput_prob>,
 'inputAlphabet': {0, 1, '⋉', '⋊'},
 'inputDist': ProbDist({('⋊', 0, 0, '⋉'): Fraction(1, 2),
           ('⋊', 0, 0, 1, '⋉'): Fraction(1, 2)}),
 'inputDist_givenPrefix': <function __main__.generateInputModel.<locals>.inputDist_givenPrefix>,
 'inputs': [('⋊', 0, 0,

The cell below defines a variety of test inputs, prefixes, and suffixes (usually restated when used later)...

In [37]:
print('TEST INPUTS:')
an_input = total_model['inputs'][1]
prefix_a = an_input[0:2]
prefix_b = an_input[0:3]
prefix_c = an_input[0:4]
prefix_d = an_input[:-2]
suffix_a = an_input[-3:]
suffix_b = tuple([1,'⋉'])
suffix_c = an_input[-2:]

print('an_input: {0}'.format(an_input))
print('prefix_a: {0}'.format(prefix_a))
print('prefix_b: {0}'.format(prefix_b))
print('prefix_c: {0}'.format(prefix_c))
print('prefix_d: {0}'.format(prefix_d))
print('suffix_a: {0}'.format(suffix_a))
print('suffix_b: {0}'.format(suffix_b))
print('suffix_c: {0}'.format(suffix_c))

TEST INPUTS:
an_input: ('⋊', 0, 0, '⋉')
prefix_a: ('⋊', 0)
prefix_b: ('⋊', 0, 0)
prefix_c: ('⋊', 0, 0, '⋉')
prefix_d: ('⋊', 0)
suffix_a: (0, 0, '⋉')
suffix_b: (1, '⋉')
suffix_c: (0, '⋉')


The two cells below show properties of the input distribution $p(X_0^f)$ and calculations derived from it...

In [38]:
print('Properties of the lexicon:')
print('p(X_0^f):')
print(inputDist)
print('inputs:')
print(total_model['inputs'])

print('input alphabet:')
print(total_model['inputAlphabet'])
print('input alphabet - edge symbols:')
total_model['inputAlphabet'] - edgeSymbols

print("<= 10 prefixes of lexicon:")
print(list(total_model['prefixes'])[0:11])

Properties of the lexicon:
p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

inputs:
[('⋊', 0, 0, 1, '⋉'), ('⋊', 0, 0, '⋉')]
input alphabet:
{0, 1, '⋊', '⋉'}
input alphabet - edge symbols:


{0, 1}

<= 10 prefixes of lexicon:
[('⋊', 0, 0, '⋉'), ('⋊', 0, 0), ('⋊',), ('⋊', 0, 0, 1, '⋉'), ('⋊', 0, 0, 1), ('⋊', 0)]


...like 
 - the probability the speaker's intended wordform has a particular prefix $p(x_0^i)$.
 - the distribution over full wordforms given a specific prefix $p(X_0^f|x_0^i)$.
 - the probability of a particular suffix given a particular prefix $p(x_{i+1}^f|x_0^i)$

In [39]:
print('Prefixes/suffixes:')

print('p(X_0^f):')
print(inputDist)
test_prefix = prefix_a
print("TEST PREFIX: {0}".format(test_prefix))
print('p(x_0^i = {0}) = {1}'.format(test_prefix, total_model['prefixProb'](test_prefix)))
print('p(X_0^f| x_0^i = {0}):'.format(test_prefix))
print(total_model['inputDist_givenPrefix'](test_prefix))
test_prefix = prefix_b
print("TEST PREFIX: {0}".format(test_prefix))
print('p(x_0^i = {0}) = {1}'.format(test_prefix, total_model['prefixProb'](test_prefix)))
print('p(X_0^f| x_0^i = {0}):'.format(test_prefix))
print(total_model['inputDist_givenPrefix'](test_prefix))
test_prefix = prefix_c
print("TEST PREFIX: {0}".format(test_prefix))
print('p(x_0^i = {0}) = {1}'.format(test_prefix, total_model['prefixProb'](test_prefix)))
print('p(X_0^f| x_0^i = {0}):'.format(test_prefix))
print(total_model['inputDist_givenPrefix'](test_prefix))

test_suffix = suffix_a
print("TEST PREFIX: {0}".format(test_prefix))
print('TEST SUFFIX: {0}'.format(test_suffix))
print('Let j = i+1:')
print(' p(x_j^f = {0}|x_0^i = {1}) = {2}'.format(test_suffix, test_prefix, total_model['suffixProb'](test_suffix, test_prefix)))
test_prefix = prefix_d
test_suffix = suffix_c
print("TEST PREFIX: {0}".format(test_prefix))
print('TEST SUFFIX: {0}'.format(test_suffix))
print('Let j = i+1:')
print(' p(x_j^f = {0}|x_0^i = {1}) = {2}'.format(test_suffix, test_prefix, total_model['suffixProb'](test_suffix, test_prefix)))


Prefixes/suffixes:
p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

TEST PREFIX: ('⋊', 0)
p(x_0^i = ('⋊', 0)) = 1
p(X_0^f| x_0^i = ('⋊', 0)):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

TEST PREFIX: ('⋊', 0, 0)
p(x_0^i = ('⋊', 0, 0)) = 1
p(X_0^f| x_0^i = ('⋊', 0, 0)):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

TEST PREFIX: ('⋊', 0, 0, '⋉')
p(x_0^i = ('⋊', 0, 0, '⋉')) = 1/2
p(X_0^f| x_0^i = ('⋊', 0, 0, '⋉')):
('⋊', 0, 0, '⋉'): 1/1 = 1.0

TEST PREFIX: ('⋊', 0, 0, '⋉')
TEST SUFFIX: (0, 0, '⋉')
Let j = i+1:
 p(x_j^f = (0, 0, '⋉')|x_0^i = ('⋊', 0, 0, '⋉')) = 0
TEST PREFIX: ('⋊', 0)
TEST SUFFIX: (0, '⋉')
Let j = i+1:
 p(x_j^f = (0, '⋉')|x_0^i = ('⋊', 0)) = 1/2


Below are the channel distribution over a single segment $p(Y|X = x)$ and the marginal distribution $p(Y)$...

In [40]:
print('Properties of the channel distribution over a single segment...')
print('p(Y_i|X_i = 0):')
# print(channelOutput_i_dist(0))
print(channelOutput_i_dist[0])
print('p(Y_i|X_i = 1):')
# print(channelOutput_i_dist(1))
print(channelOutput_i_dist[1])

print("p(Y_i):")
print(total_model['channelOutput_i_marginal_dist'])

Properties of the channel distribution over a single segment...
p(Y_i|X_i = 0):
0: 0.85
1: 0.15

p(Y_i|X_i = 1):
0: 0.15
1: 0.85

p(Y_i):
0: 0.25
1: 0.25
'⋊': 0.25
'⋉': 0.25



Below are example calculations related to the following channel distribution over prefixes:
 - $p(y_0^i|x_0^i)$
 - $p(y_0^j|x_0^i)$

In [41]:
print('Properties of the channel distribution over prefixes...')

print("p(X_0^f):")
print(inputDist)
  
test_input_full = an_input
test_output_full = an_input
print("TEST INPUT full: {0}".format(test_input_full))
print("TEST OUTPUT full: {0}".format(test_output_full))
print('p(y_0^i = {0}| x_0^i = {1}) = {2}'.format(test_output_full, test_input_full, total_model['channelOutput_prefix_prob'](test_input_full, test_output_full)))
print('len({0}) = {1}'.format(trimBoundariesFromSequence(test_input_full), len(trimBoundariesFromSequence(test_input_full))))
print('p(no_error_i) = {0}'.format(pNoError_i))
print('{2}^(len({0})) = {1}'.format(trimBoundariesFromSequence(test_input_full), pNoError_i ** len(trimBoundariesFromSequence(test_input_full)), pNoError_i))

print(' ')
test_input_slice = prefix_a
test_output_slice = prefix_a
print("TEST INPUT prefix: {0}".format(test_input_slice))
print("TEST OUTPUT prefix: {0}".format(test_output_slice))
print('p(y_0^i = {0}| x_0^i = {1}) = {2}'.format(test_output_slice, test_input_slice, total_model['channelOutput_prefix_prob'](test_input_slice, test_output_slice)))

print(' ')
test_input_slice = prefix_a
test_output_slice = prefix_b
print("TEST INPUT prefix: {0}".format(test_input_slice))
print("TEST OUTPUT prefix: {0}".format(test_output_slice))
print('p(y_0^j = {0}| x_0^i = {1}) = {2}'.format(test_output_slice, test_input_slice, total_model['channelOutput_prefix_prob'](test_input_slice, test_output_slice)))

print(' ')
test_input_slice = prefix_b
test_output_slice = prefix_a
print("TEST INPUT prefix: {0}".format(test_input_slice))
print("TEST OUTPUT prefix: {0}".format(test_output_slice))
print('p(y_0^j = {0}| x_0^i = {1}) = {2}'.format(test_output_slice, test_input_slice, total_model['channelOutput_prefix_prob'](test_input_slice, test_output_slice)))

Properties of the channel distribution over prefixes...
p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

TEST INPUT full: ('⋊', 0, 0, '⋉')
TEST OUTPUT full: ('⋊', 0, 0, '⋉')
p(y_0^i = ('⋊', 0, 0, '⋉')| x_0^i = ('⋊', 0, 0, '⋉')) = 0.7224999999999999
len((0, 0)) = 2
p(no_error_i) = 0.75
0.75^(len((0, 0))) = 0.5625
 
TEST INPUT prefix: ('⋊', 0)
TEST OUTPUT prefix: ('⋊', 0)
p(y_0^i = ('⋊', 0)| x_0^i = ('⋊', 0)) = 0.85
 
TEST INPUT prefix: ('⋊', 0)
TEST OUTPUT prefix: ('⋊', 0, 0)
p(y_0^j = ('⋊', 0, 0)| x_0^i = ('⋊', 0)) = 0.7224999999999999
 
TEST INPUT prefix: ('⋊', 0, 0)
TEST OUTPUT prefix: ('⋊', 0)
p(y_0^j = ('⋊', 0)| x_0^i = ('⋊', 0, 0)) = 0.85


Below are samples from $p(Y_0^i|x_0^i)$:

In [42]:
print("TEST INPUT prefix: {0}".format(prefix_b))
print('10 samples from p(Y_0^i|x_0^i = {0}): {1}'.format(prefix_b, list(sampleFrom(lambda : total_model['channelOutput_prefix_sampler'](prefix_b), 10)) ))

TEST INPUT prefix: ('⋊', 0, 0)
10 samples from p(Y_0^i|x_0^i = ('⋊', 0, 0)): [('⋊', 0, 0), ('⋊', 0, 0), ('⋊', 0, 1), ('⋊', 0, 0), ('⋊', 0, 1), ('⋊', 0, 0), ('⋊', 0, 0), ('⋊', 0, 0), ('⋊', 0, 0), ('⋊', 1, 0)]


Below is an example calculation of $p(y_0^i)$:

In [43]:
print('TEST OUTPUT prefix = {0}'.format(prefix_b))
print('p(y_0^i = {0}) = {1}'.format(prefix_b, total_model['channelOutput_prefix_marginal_prob'](prefix_b)))

TEST OUTPUT prefix = ('⋊', 0, 0)
p(y_0^i = ('⋊', 0, 0)) = 0.7224999999999999


Finally, below are example calculations of $p(\hat{x}_0^j| x_0^i)$:

In [44]:
print("p(X_0^f):")
print(inputDist)

print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(('⋊', 1), prefix_b, total_model['est_channelInput_prob'](prefix_b, ('⋊', 1))))
print(' ')

print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(prefix_b, prefix_b, total_model['est_channelInput_prob'](prefix_b, prefix_b)))
print(' ')
print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(an_input, prefix_b, total_model['est_channelInput_prob'](prefix_b, an_input)))
print(' ')
print('est p(x-hat_0^j = {0}| x_0^i = {1}) = {2}'.format(prefix_b, an_input, total_model['est_channelInput_prob'](an_input, prefix_b)))

p(X_0^f):
('⋊', 0, 0, 1, '⋉'): 1/2 = 0.5
('⋊', 0, 0, '⋉'): 1/2 = 0.5

est p(x-hat_0^j = ('⋊', 1)| x_0^i = ('⋊', 0, 0)) = 0
 
est p(x-hat_0^j = ('⋊', 0, 0)| x_0^i = ('⋊', 0, 0)) = 1.0
 
est p(x-hat_0^j = ('⋊', 0, 0, '⋉')| x_0^i = ('⋊', 0, 0)) = 0.5
 
est p(x-hat_0^j = ('⋊', 0, 0)| x_0^i = ('⋊', 0, 0, '⋉')) = 1.0
